In [9]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='https://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('https://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\", sans-serif;\r\n    font-size: 100%;\r\n}\r\n\r\n</style>")

In [10]:
import Chess
using NBInclude

In [11]:
@nbinclude "3.1 - Evaluation.ipynb"

evaluateEPCaptureScoreDelta (generic function with 1 method)

## Memoisierung - MiniMax

Hier wird der MiniMax-Algorithmus in Verbindung mit Meomisierung implementiert. Die Idee ist, dass dank der Speicherung von bereits berechneten Stellungen und deren Wertigkeiten, weitere Berechnungen schneller durchgeführt werden können. Die resultierende KI kann somit in gleicher Zeit eine größere Suchtiefe erreichen und damit genauere und bessere Züge berechnen.

Die Funktion `getPositionScoreMemo` berechnet den Score einer Stellung unter Verwendung der Transpositionstabelle zur Aufrufung von bekannten und Speicherung von neuen Stellungen samt Wertigkeiten.

**Input**:
+ extboard &rarr; der aktuelle Spielstand der Berechnung
+ func &rarr; der nächste Berechnungsschritt (maxMemo / minMemo)
+ depth &rarr; Suchtiefe (maximale Suchtiefe - aktuelle Suchtiefe)
+ transpositions &rarr; die Transpositionstabelle

**Output**:
+ tScore &rarr; der Score des nächsten Berechnungsschritts (maximiert / minimiert)

In [12]:
function getPositionScoreMemo(
        extboard::ExtendedBoard, func::Function, depth::Int64, 
        transpositions::Dict{UInt64, Transposition})
    transposition = get(transpositions, extboard.zobrist.hash, nothing)
    if transposition != nothing
        tDepth, tScore = transposition
        if tDepth >= depth
            #global hits += 1
            return tScore
        end
    end
    #global misses += 1
    tScore = func(extboard, depth - 1, transpositions)
    transpositions[extboard.zobrist.hash] = (depth, tScore)
    return tScore
end

getPositionScoreMemo (generic function with 1 method)

Die Funktion `memoMax` ist die `minimaxMax` Funktion unter Verwendung der Transpositionstabelle zur Aufrufung von bekannten und Speicherung von neuen Stellungen samt Wertigkeiten.

**Input**:
+ extboard &rarr; der aktuelle Spielstand der Berechnung
+ depth &rarr;  Suchtiefe (maximale Suchtiefe - aktuelle Suchtiefe)
+ transpositions &rarr; die Transpositionstabelle

**Output**:
+ maxValue &rarr; der maximierte, bestmögliche Score der aktuellen Suche

In [13]:
function memoMax(extboard::ExtendedBoard, depth::Int64, transpositions::Dict{UInt64, Transposition})::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) - depth
    elseif depth == 0
        return extboard.score
    end
    maxValue = typemin(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = getPositionScoreMemo(extboard, memoMin, depth, transpositions)
        undomove!(extboard, undo)
        maxValue = max(value, maxValue)
    end
    Chess.recycle!(legalMoves)
    return maxValue
end

memoMax (generic function with 1 method)

Die Funktion `memoMin` ist die `minimaxMin` Funktion unter Verwendung der Transpositionstabelle zur Aufrufung von bekannten und Speicherung von neuen Stellungen samt Wertigkeiten.

**Input**:
+ extboard &rarr; der aktuelle Spielstand der Berechnung
+ depth &rarr; Suchtiefe (maximale Suchtiefe - aktuelle Suchtiefe)
+ transpositions &rarr; die Transpositionstabelle

**Output**:
+ minValue &rarr; der minimierte, bestmögliche Score der aktuellen Suche

In [14]:
function memoMin(extboard::ExtendedBoard, depth::Int64, transpositions::Dict{UInt64, Transposition})::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) + depth
    elseif depth == 0
        return extboard.score
    end
    minValue = typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = getPositionScoreMemo(extboard, memoMax, depth, transpositions)
        undomove!(extboard, undo)
        minValue = min(value, minValue)
    end
    Chess.recycle!(legalMoves)
    return minValue
end

memoMin (generic function with 1 method)

Die Funktion `minimaxMemo` ist die `minimax` Funktion unter Verwendung der Transpositionstabelle zur Aufrufung von bekannten und Speicherung von neuen Stellungen samt Wertigkeiten. 

**Input**:
+ extboard &rarr; der aktuelle Spielstand
+ depth &rarr; die gewünschte Suchtiefe 

**Output**:
+ minmaxValue &rarr; der bestmögliche Score unter der gewünschten Suchtiefe
+ scoredMoves[minmaxValue] &rarr; alle Züge mit dem bestmöglichen Score

Sie gibt ein Array an bestmöglichen Zügen aus. Die `MemoAI` wählt anschließend zufällig einen Zug aus. 

In [15]:
function minimaxMemo(extboard::ExtendedBoard, depth::Int64)::Tuple{Int32, Vector{Chess.Move}}
    legalMoves = Chess.moves(extboard.board)
    transpositions = Dict{UInt64, Transposition}()
    sideIsWhite = Chess.sidetomove(extboard.board) == Chess.WHITE
    scoredMoves = Dict{Int32, Vector{Chess.Move}}()
    minmaxValue = sideIsWhite ? typemin(Int32) : typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        if sideIsWhite
            value = memoMin(extboard, depth - 1, transpositions)
            minmaxValue = max(value, minmaxValue)
        else
            value = memoMax(extboard, depth - 1, transpositions)
            minmaxValue = min(value, minmaxValue)
        end
        movesWithSameScore = get(scoredMoves, value, Chess.Move[])
        push!(movesWithSameScore, move)
        scoredMoves[value] = movesWithSameScore
        println("$(value) $(move)")
        undomove!(extboard, undo)
    end
    return minmaxValue, scoredMoves[minmaxValue]
end

minimaxMemoAll (generic function with 1 method)

***